In [1]:
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import Precision
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import funzioni as fx
import os


def definizione_parametri_e_target(df):
    df['MinMaxChange'] = df["MaxMinRel"]
    X = df[[
        "Volume", 
        "EMA_20",
        "EMA_50",
        "PSARaf",
        "PSARr", 
        "PSAR",
        "MACD",
        "MACDh",
        "TSI",
        "TSIs",
        "SUPERT",
        "SUPERTd",
        "DM_OSC",
        "TRIX",
        "TRIXs",
        "VTX_OSC",
        "AROONOSC",
        "VI_OSC",
        "ATR"
    ]].copy()
    return X



def elimina_valori_non_validi(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True, axis=0)
    return df


def addestramento(model, dati_ticker):
    giorni_previsione = 20

    dati_ticker = elimina_valori_non_validi(dati_ticker)
    dati_ticker = definizione_parametri_e_target(dati_ticker)
    
    
    target = dati_ticker["EMA_20"].shift(-giorni_previsione)

    # 2. Creazione di Sequenze Temporali
    look_back = 30
    X, Y = [], []
    for i in range(look_back, len(dati_ticker) - giorni_previsione):
        X.append(dati_ticker.iloc[i - look_back:i].values)
        Y.append(target[i])
    X, Y = np.array(X), np.array(Y)

    # 3. Creazione del Modello
    if model == 0:
        model = keras.Sequential([
            layers.LSTM(200, input_shape=(X.shape[1], X.shape[2]), return_sequences=True),
            layers.LSTM(200),
            layers.Dense(1, activation='sigmoid')
        ])

    # 4. Allenamento del Modello
    model.compile(optimizer='adam', loss='log_cosh', metrics=['mean_absolute_error'])
    model.fit(X, Y, epochs=50, batch_size=32, validation_split=0.2)

    # 5. Valutazione e Test
    test_loss, mean_absolute_error = model.evaluate(X, Y)
    print(f"\033[42mTest Loss: {test_loss}\033[0m")
    print(f"\033[42mMean Absolute Error: {mean_absolute_error}\033[0m")
    
    return model

In [2]:
lista_file = os.listdir("../Dati_borsa/Dati_calcolati_scalati")
tot_file = len(lista_file)
tickers = fx.tickers_de_giro("../DeGiro con formule.xlsx")

if os.path.exists('LSTM.keras'):
    model = keras.models.load_model('LSTM.keras')
else:
    model = 0

# Load the last ticker processed if the file exists.
last_ticker_processed = None
if os.path.exists('last_ticker.txt'):
    with open('last_ticker.txt', 'r') as f:
        last_ticker_processed = f.read().strip()

start_from_next_ticker = last_ticker_processed is None

i = 0
for f in lista_file:
    if not start_from_next_ticker:
        if f == last_ticker_processed:
            start_from_next_ticker = True
        print(f"\r{i}) Caricamento ticker {f}                           ", end=' ', flush=True)
        i += 1
        continue
    nome_file, estensione = os.path.splitext(f)   
    categoria = tickers.loc[tickers["Ticker"] == nome_file, "Categoria"].iloc[0]
    print(f"\033[41m{i} di {tot_file}) Caricamento ticker {nome_file}                                       \033[0m", flush=True)
    df = pd.read_parquet(f"../Dati_borsa/Dati_calcolati_scalati/{f}")
    if (len(df) > 0) and (categoria != "D"):
        model = addestramento(model, df)
    i += 1
    
    with open('last_ticker.txt', 'w') as file_ticker:
        file_ticker.write(f)
    model.save("LSTM.keras")
    
os.remove("last_ticker.txt")
print("\nFine.")

0 di 4590) Caricamento ticker A                                        Epoch 1/50
112/112 [==============================] - 23s 129ms/step - loss: 15.3887 - mean_absolute_error: 16.0585 - val_loss: 90.3418 - val_mean_absolute_error: 91.0350
Epoch 2/50
112/112 [==============================] - 12s 108ms/step - loss: 12.1371 - mean_absolute_error: 12.8092 - val_loss: 89.1978 - val_mean_absolute_error: 89.8909
Epoch 3/50
112/112 [==============================] - 12s 111ms/step - loss: 12.1328 - mean_absolute_error: 12.8017 - val_loss: 89.3195 - val_mean_absolute_error: 90.0127
Epoch 4/50
112/112 [==============================] - 14s 128ms/step - loss: 12.1322 - mean_absolute_error: 12.8011 - val_loss: 89.6313 - val_mean_absolute_error: 90.3245
Epoch 5/50
112/112 [==============================] - 12s 108ms/step - loss: 12.1310 - mean_absolute_error: 12.8007 - val_loss: 88.9182 - val_mean_absolute_error: 89.6113
Epoch 6/50
112/112 [==============================] - 13s 114ms/step - los

112/112 [==============================] - 12s 103ms/step - loss: 15.4861 - mean_absolute_error: 16.1464 - val_loss: 16.7040 - val_mean_absolute_error: 17.3932
Epoch 45/50
112/112 [==============================] - 12s 104ms/step - loss: 15.4487 - mean_absolute_error: 16.1092 - val_loss: 16.4978 - val_mean_absolute_error: 17.1827
Epoch 46/50
112/112 [==============================] - 12s 105ms/step - loss: 15.4320 - mean_absolute_error: 16.0927 - val_loss: 16.5377 - val_mean_absolute_error: 17.2256
Epoch 47/50
112/112 [==============================] - 12s 103ms/step - loss: 15.3276 - mean_absolute_error: 15.9840 - val_loss: 16.4679 - val_mean_absolute_error: 17.1562
Epoch 48/50
112/112 [==============================] - 12s 106ms/step - loss: 15.3257 - mean_absolute_error: 15.9839 - val_loss: 16.7338 - val_mean_absolute_error: 17.4224
Epoch 49/50
112/112 [==============================] - 12s 105ms/step - loss: 15.2211 - mean_absolute_error: 15.8805 - val_loss: 16.9058 - val_mean_abso

12/12 [==============================] - 1s 102ms/step - loss: 3.4737 - mean_absolute_error: 4.0306 - val_loss: 13.4338 - val_mean_absolute_error: 14.1269
Epoch 38/50
12/12 [==============================] - 1s 107ms/step - loss: 3.4770 - mean_absolute_error: 4.0351 - val_loss: 13.7252 - val_mean_absolute_error: 14.4183
Epoch 39/50
12/12 [==============================] - 1s 101ms/step - loss: 3.4521 - mean_absolute_error: 3.9877 - val_loss: 13.7964 - val_mean_absolute_error: 14.4895
Epoch 40/50
12/12 [==============================] - 1s 111ms/step - loss: 3.4541 - mean_absolute_error: 3.9941 - val_loss: 13.5293 - val_mean_absolute_error: 14.2225
Epoch 41/50
12/12 [==============================] - 1s 101ms/step - loss: 3.4529 - mean_absolute_error: 3.9944 - val_loss: 13.8339 - val_mean_absolute_error: 14.5270
Epoch 42/50
12/12 [==============================] - 1s 110ms/step - loss: 3.4478 - mean_absolute_error: 3.9818 - val_loss: 13.9945 - val_mean_absolute_error: 14.6876
Epoch 43/5

Epoch 34/50
112/112 [==============================] - 11s 100ms/step - loss: 8.9118 - mean_absolute_error: 9.5447 - val_loss: 47.1673 - val_mean_absolute_error: 47.8605
Epoch 35/50
112/112 [==============================] - 11s 102ms/step - loss: 9.2669 - mean_absolute_error: 9.9097 - val_loss: 50.9109 - val_mean_absolute_error: 51.6041
Epoch 36/50
112/112 [==============================] - 12s 107ms/step - loss: 9.1464 - mean_absolute_error: 9.7873 - val_loss: 50.5767 - val_mean_absolute_error: 51.2698
Epoch 37/50
112/112 [==============================] - 12s 103ms/step - loss: 8.9143 - mean_absolute_error: 9.5533 - val_loss: 49.5076 - val_mean_absolute_error: 50.2007
Epoch 38/50
112/112 [==============================] - 12s 104ms/step - loss: 8.8200 - mean_absolute_error: 9.4518 - val_loss: 49.9862 - val_mean_absolute_error: 50.6794
Epoch 39/50
112/112 [==============================] - 11s 102ms/step - loss: 8.7804 - mean_absolute_error: 9.4116 - val_loss: 48.8400 - val_mean_abso


KeyboardInterrupt

